<a href="https://colab.research.google.com/github/brycecturner/ship_detection_from_space/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt


random.seed(108)

In [19]:
os.chdir('/content/drive/MyDrive/ship_detection/')

In [32]:
#!git clone https://github.com/ultralytics/yolov5 /content/drive/MyDrive/ship_detection/yolov5 # clone

In [31]:
# !pip install -r requirements.txt

In [ ]:
def extract_info_from_xml(file_dir, using_file):

  root = ET.parse(file_dir + using_file).getroot()

  info_dict = {}

  for elem in root:
    # print(elem.tag)
    if elem.tag == 'filename':
      info_dict['filename'] = elem.text

    for subelem in elem:

      #print(subelem.tag)
      if subelem.tag == 'name':
        info_dict['class'] = subelem.text
      if subelem.tag == 'bndbox':
        box_dict = {}
        for i in subelem:
          box_dict[i.tag] = int(i.text)
        info_dict['bndbox'] = box_dict

  return info_dict

In [ ]:
def convert_to_yolov5(info_dict):
    print_buffer = []
    try:
      class_id = classname_to_id_map[info_dict["class"]]
    except KeyError:
      print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        

    # Transform the bbox co-ordinates as per the format required by YOLO v5
    b_center_x = (info_dict['bndbox']["xmin"] + info_dict['bndbox']["xmax"]) / 2 
    b_center_y = (info_dict['bndbox']["ymin"] + info_dict['bndbox']["ymax"]) / 2
    b_width    = (info_dict['bndbox']["xmax"] - info_dict['bndbox']["xmin"])
    b_height   = (info_dict['bndbox']["ymax"] - info_dict['bndbox']["ymin"])
    b_center_x /= 512 
    b_center_y /= 512 
    b_width    /= 512 
    b_height   /= 512 
    #Write the bbox details to the file 
    print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("drive/MyDrive/ship_detection/MASATI-v2/annotations", info_dict["filename"].replace("png", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
# For the purposes of this work, it does not matter whether the ship is near the coast or not
# therefore, flatten everything into a single class
classname_to_id_map = {'ship' : 0,
                       'coast_ship' : 0}

In [ ]:
file_dir = '/content/drive/MyDrive/ship_detection/MASATI-v2/coast_ship/'
using_files = os.listdir(file_dir)
using_files[0]

In [ ]:
# This converts the xml files into the requisite yolo format
# all_info = [extract_info_from_xml(file_dir, x) for x in using_files]
#[convert_to_yolov5(x) for x in all_info]

In [ ]:
os.chdir('/content/drive/MyDrive/ship_detection/yolov5')
!python train.py --img 512 --epochs 3 --data ship_data.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=ship_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
requirements: /content/drive/MyDrive/ship_detection/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.11 torch-2.0.0+cu118 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, i

In [8]:
os.chdir('/content/drive/MyDrive/ship_detection/yolov5')
!python detect.py --weights runs/train/exp/weights/last.pt --img 512 --conf 0.4 --source ../yolo_data/test/

detect: weights=['runs/train/exp/weights/last.pt'], source=../yolo_data/test/, data=data/coco128.yaml, imgsz=[512, 512], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: /content/drive/MyDrive/ship_detection/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1192 /content/drive/MyDrive/ship_detection/yolo_data/test/c0005.png: 512x512 (no detections), 315.0ms
image 2/1192 /content/drive/MyDrive/ship_detection/yolo_data/test/c0006.png: 512x512 (no detections), 245.4ms
image 3/1192 /content/drive/MyDrive/ship_detection/yolo_da

In [18]:
import re
import numpy as np

with open("runs/detect/text_results/detect_exp4.txt", 'r') as f:
  data = f.read().splitlines()

detection_results = [' '.join(l.split(" ")[4:6]) for l in data]
np.unique(detection_results)

array(['(no detections),', '512x512 (no'], dtype='<U16')

Note that all values for the training data result in '(no detections)'. The single outlier is an artifact of the name of the file and is therefore not being parsed correctly.